In [1]:
import pandas as pd
import cupy as cp


c:\Users\Ethan K\OneDrive\Desktop\School\2.2\sciencific_computing\.venv\Lib\site-packages\cupy\_environment.py:217: UserWarning: CUDA path could not be detected. Set CUDA_PATH environment variable if CuPy fails to load.
  warnings.warn(


In [2]:
df = pd.read_csv('week1.csv')
df

,age,sex,lat,long,employment,infected
0,0,m,54.522510,-1.571896,U,False
1,0,m,54.554030,-1.524968,U,True
2,0,m,54.552486,-1.435203,U,False
3,0,m,54.537189,-1.566215,U,True
4,0,m,54.528212,-1.588462,U,False
5,90,f,51.634416,-2.925863,V,True
6,90,f,51.556972,-3.036290,V,True
7,90,f,51.588992,-2.921915,V,False
8,90,f,51.590974,-2.954539,V,False
9,90,f,51.576716,-2.952142,V,False


In [3]:
infected_df = df[df['infected']]
infected_df = infected_df.reset_index(drop=True)
infected_df

,age,sex,lat,long,employment,infected
0,0,m,54.554030,-1.524968,U,True
1,0,m,54.537189,-1.566215,U,True
2,90,f,51.634416,-2.925863,V,True
3,90,f,51.556972,-3.036290,V,True


In [4]:

def latlong2osgbgrid_cupy(lat, long, input_degrees=True):
    '''
    Converts latitude and longitude (ellipsoidal) coordinates into northing and easting (grid) coordinates, using a Transverse Mercator projection.
    
    Inputs:
    lat: latitude coordinate (N)
    long: longitude coordinate (E)
    input_degrees: if True (default), interprets the coordinates as degrees; otherwise, interprets coordinates as radians
    
    Output:
    (northing, easting)
    '''
    
    if input_degrees:
        lat = lat * cp.pi/180
        long = long * cp.pi/180

    a = 6377563.396
    b = 6356256.909
    e2 = (a**2 - b**2) / a**2

    N0 = -100000 # northing of true origin
    E0 = 400000 # easting of true origin
    F0 = .9996012717 # scale factor on central meridian
    phi0 = 49 * cp.pi / 180 # latitude of true origin
    lambda0 = -2 * cp.pi / 180 # longitude of true origin and central meridian
    
    sinlat = cp.sin(lat)
    coslat = cp.cos(lat)
    tanlat = cp.tan(lat)
    
    latdiff = lat-phi0
    longdiff = long-lambda0

    n = (a-b) / (a+b)
    nu = a * F0 * (1 - e2 * sinlat ** 2) ** -.5
    rho = a * F0 * (1 - e2) * (1 - e2 * sinlat ** 2) ** -1.5
    eta2 = nu / rho - 1
    M = b * F0 * ((1 + n + 5/4 * (n**2 + n**3)) * latdiff - 
                  (3*(n+n**2) + 21/8 * n**3) * cp.sin(latdiff) * cp.cos(lat+phi0) +
                  15/8 * (n**2 + n**3) * cp.sin(2*(latdiff)) * cp.cos(2*(lat+phi0)) - 
                  35/24 * n**3 * cp.sin(3*(latdiff)) * cp.cos(3*(lat+phi0)))
    I = M + N0
    II = nu/2 * sinlat * coslat
    III = nu/24 * sinlat * coslat ** 3 * (5 - tanlat ** 2 + 9 * eta2)
    IIIA = nu/720 * sinlat * coslat ** 5 * (61-58 * tanlat**2 + tanlat**4)
    IV = nu * coslat
    V = nu / 6 * coslat**3 * (nu/rho - cp.tan(lat)**2)
    VI = nu / 120 * coslat ** 5 * (5 - 18 * tanlat**2 + tanlat**4 + 14 * eta2 - 58 * tanlat**2 * eta2)

    northing = I + II * longdiff**2 + III * longdiff**4 + IIIA * longdiff**6
    easting = E0 + IV * longdiff + V * longdiff**3 + VI * longdiff**5

    return(northing, easting)

In [5]:
cupy_lat = cp.asarray(infected_df['lat'])
cupy_long = cp.asarray(infected_df['long'])

northing_cp, easting_cp = latlong2osgbgrid_cupy(cupy_lat, cupy_long)


infected_df  # Display the DataFrame


RuntimeError: CuPy failed to load nvrtc64_120_0.dll: FileNotFoundError: Could not find module 'nvrtc64_120_0.dll' (or one of its dependencies). Try using the full path with constructor syntax.